In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('/content/POP.csv', encoding='ISO-8859-1')

In [7]:
df

,country,id,name,population
0,IND,1,India,"1,417,173"
1,CHN,2,China,"1,412,175"
2,USA,3,United States,"333,288"
3,IDN,4,Indonesia,"275,501"
4,PAK,5,Pakistan,"235,825"
...,...,...,...,...
227,SSF,228,Sub-Saharan Africa,"1,211,190"
228,LIC,229,Low income,"703,728"
229,LMC,230,Lower middle income,"3,190,184"
230,UMC,231,Upper middle income,"2,784,368"


In [8]:
df.columns

Index(['country', 'id', 'name', 'population'], dtype='object')

In [12]:
import sqlite3
import pandas as pd

def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return None

def create_tables(conn):
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS country_pop (
            name TEXT,
            country TEXT,
            id INTEGER,
            population REAL
        )
    ''')
    conn.commit()

def load_country_pop(conn, dataset_file):
    cursor = conn.cursor()
    dataset = pd.read_csv('/content/POP.csv', encoding='ISO-8859-1')
     # Check the column names
    print(dataset.columns)
    # Or you can print the first few rows to inspect the data
    print(dataset.head())
    # Extract relevant columns for country population table
    country_pop = dataset[['country', 'id', 'name', 'population']]

    conn.commit()

def main():
    database = "data_warehouse.db"
    dataset_file = "/content/POP.csv"

    conn = create_connection(database)
    if conn is not None:
        create_tables(conn)
        load_country_pop(conn, dataset_file)
        print("Data warehouse created successfully.")
        conn.close()
    else:
        print("Error! Cannot create the database connection.")

if __name__ == '__main__':
    main()

Index(['country', 'id', 'name', 'population'], dtype='object')
  country  id           name population
0     IND   1          India  1,417,173
1     CHN   2          China  1,412,175
2     USA   3  United States    333,288
3     IDN   4      Indonesia    275,501
4     PAK   5       Pakistan    235,825
Data warehouse created successfully.


In [13]:
import sqlite3

# Function to create a connection to the SQLite database
def create_connection(db_file):
    try:
        return sqlite3.connect(db_file)
    except sqlite3.Error as e:
        print(f"Error creating connection: {e}")
        return None

# Function to create University table
def create_university_table(conn):
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS University (
            id INTEGER PRIMARY KEY,
            Name TEXT NOT NULL,
            Private TEXT NOT NULL,
            Apps INTEGER NOT NULL,
            Accept INTEGER NOT NULL,
            Enroll INTEGER NOT NULL,
            Top10perc INTEGER NOT NULL,
            Top25perc INTEGER NOT NULL,
            F_Undergrad INTEGER NOT NULL,
            p_Undergrad INTEGER NOT NULL,
            Outstate INTEGER NOT NULL,
            Room_Board INTEGER NOT NULL,
            Books INTEGER NOT NULL,
            Personal INTEGER NOT NULL,
            PhD INTEGER NOT NULL,
            Terminal INTEGER NOT NULL,
            S_F_Ratio REAL NOT NULL,
            perc_alumni INTEGER NOT NULL,
            Expend INTEGER NOT NULL,
            Grad_Rate INTEGER NOT NULL,
            UNIQUE(Name)
        )
    ''')

    conn.commit()

def main():
    database = "data_warehouse.db"

    # Create a database connection
    conn = create_connection(database)
    if conn is not None:
        # Create tables
        create_tables(conn)
        print("Tables created successfully.")
        conn.close()
    else:
        print("Error! Cannot create the database connection.")

if __name__ == '__main__':
    main()

Table created successfully.


In [14]:
import sqlite3
import pandas as pd
from tabulate import tabulate

# Function to create a connection to the SQLite database
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return None

def create_country_fact_table(conn):
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Country_Fact (
            id INTEGER PRIMARY KEY,
            country TEXT,
            name TEXT,
            population INTEGER
        )
    ''')

    conn.commit()

def populate_country_fact_table(conn, df):
    cursor = conn.cursor()

    # Populating the fact table based on existing data and relationships
    for index, row in df.iterrows():
        cursor.execute('''
            INSERT INTO Country_Fact (id, country, name, population)
            VALUES (?, ?, ?, ?)
        ''', (row['id'], row['country'], row['name'], row['population']))

    conn.commit()

def display_country_fact_table(conn):
    cursor = conn.cursor()

    cursor.execute('''
        SELECT * FROM Country_Fact
    ''')

    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=['ID', 'Country', 'Name', 'Population'])
    print("Country Fact Table:")
    print(tabulate(df.head(), headers='keys', tablefmt='pretty'))

def main():
    database = "country_data.db"

    # Create a database connection
    conn = create_connection(database)
    if conn is not None:
        # Create the Fact table
        create_country_fact_table(conn)

        # Read the dataset
        df = pd.DataFrame({'country': ['IND', 'CHN', 'USA', 'IDN', 'PAK'],
                           'id': [1, 2, 3, 4, 5],
                           'name': ['India', 'China', 'United States', 'Indonesia', 'Pakistan'],
                           'population': [1417173, 1412175, 333288, 275501, 235825]})

        # Populate the Fact table
        populate_country_fact_table(conn, df)

        # Display the Fact table
        display_country_fact_table(conn)

        conn.close()
    else:
        print("Error! Cannot create the database connection.")

if __name__ == '__main__':
    main()

Country Fact Table:
+---+----+---------+---------------+------------+
|   | ID | Country |     Name      | Population |
+---+----+---------+---------------+------------+
| 0 | 1  |   IND   |     India     |  1417173   |
| 1 | 2  |   CHN   |     China     |  1412175   |
| 2 | 3  |   USA   | United States |   333288   |
| 3 | 4  |   IDN   |   Indonesia   |   275501   |
| 4 | 5  |   PAK   |   Pakistan    |   235825   |
+---+----+---------+---------------+------------+
